In [4]:
import cv2
import mediapipe as mp
import sys
import serial

arduino = serial.Serial('/dev/tty.usbmodem1101', 9600)

# MediaPipe 초기화
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh()

# 웹캠 켜기
cap = cv2.VideoCapture(0)

# 창 이름 설정
window_name = 'Posture Detection'
cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)

# 폰트 설정
font = cv2.FONT_HERSHEY_DUPLEX

baseline_chin_y = None
baseline_ear_z = None
measured = False

# 튜닝 가능한 값
chin_y_threshold = 20  # 턱 Y좌표 기준 변화량
ear_z_threshold = 0.02  # 귀 Z좌표 기준 변화량

while cap.isOpened():
    success, image = cap.read()
    if not success:
        break

    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(image_rgb)
    image_height, image_width, _ = image.shape

    key = cv2.waitKey(5)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            # 랜드마크 추출
            left_ear = face_landmarks.landmark[33]
            right_ear = face_landmarks.landmark[263]
            chin = face_landmarks.landmark[152]

            # 좌표 변환
            left_ear_pos = (int(left_ear.x * image_width), int(left_ear.y * image_height))
            right_ear_pos = (int(right_ear.x * image_width), int(right_ear.y * image_height))
            chin_pos = (int(chin.x * image_width), int(chin.y * image_height))

            chin_y = chin_pos[1]
            ear_z = (left_ear.z + right_ear.z) / 2  # 양쪽 귀 평균 z값

            # 얼굴 랜드마크 표시
            cv2.circle(image, left_ear_pos, 4, (0, 255, 0), -1)
            cv2.circle(image, right_ear_pos, 4, (0, 255, 0), -1)
            cv2.circle(image, chin_pos, 4, (0, 0, 255), -1)

            # 텍스트 출력
            cv2.putText(image, f'Chin Y: {chin_y:.0f}', (20, 40),
                        font, 0.8, (50, 150, 255), 2)

            if not measured:
                cv2.putText(image, "Sit upright and press 'S' to save baseline", (20, 80),
                            font, 0.7, (0, 0, 255), 2)

                if key == ord('s'):
                    baseline_chin_y = chin_y
                    baseline_ear_z = ear_z
                    measured = True
                    print("✅ 기준 자세 저장 완료!")
            else:
                chin_y_diff = chin_y - baseline_chin_y
                ear_z_diff = baseline_ear_z - ear_z

                chin_score = min(max((chin_y_diff / chin_y_threshold) * 100, 0), 100)
                ear_score = min(max((ear_z_diff / ear_z_threshold) * 100, 0), 100)

                score = max(chin_score, ear_score)

                cv2.putText(image, f'Turtle Neck Score: {score:.1f}/100', (20, 80),
                            font, 0.8, (0, 100, 255), 2)
                if score > 50:
                    cv2.putText(image, "Warning: Please fix your posture!", (20, 120),
                    font, 0.8, (0, 0, 255), 2)
                    arduino.write(b'1')
                else:
                    arduino.write(b'0')


    # 화면 출력
    cv2.imshow(window_name, image)

    if key == 27:  # ESC 누르면 종료
        break

# 종료 처리
cap.release()
cv2.destroyWindow(window_name)
cv2.waitKey(1)
sys.exit()


I0000 00:00:1745903081.617236 9680029 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M2
W0000 00:00:1745903081.621119 9682140 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1745903081.626040 9682140 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


✅ 기준 자세 저장 완료!


2025-04-29 14:05:23.955 python[83036:9680029] _TIPropertyValueIsValid called with 16 on nil context!
2025-04-29 14:05:23.955 python[83036:9680029] imkxpc_getApplicationProperty:reply: called with incorrect property value 16, bailing.
2025-04-29 14:05:23.955 python[83036:9680029] Text input context does not respond to _valueForTIProperty:
2025-04-29 14:05:26.688 python[83036:9680029] _TIPropertyValueIsValid called with 16 on nil context!
2025-04-29 14:05:26.688 python[83036:9680029] imkxpc_getApplicationProperty:reply: called with incorrect property value 16, bailing.
2025-04-29 14:05:26.688 python[83036:9680029] Text input context does not respond to _valueForTIProperty:


SystemExit: 